In [ ]:
import pandas as pd


df = pd.read('health_train.csv')

X = df.drop(columns=['stroke'])
y = df['stroke']

In [ ]:
from sklearn.model_selection import StratifiedKFold


results_df = pd.DataFrame(
    {
        "Model": [],
        "Accuracy": [],
        "Recall": [],
        "ROC-AUC": [],
        "PR-AUC": [],
    }
).astype(
    {
        "Model": str,
        "Accuracy": float,
        "Recall": float,
        "ROC-AUC": float,
        "PR-AUC": float,
    }
)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)